In [1]:
from copy import deepcopy
import yaml
from pathlib import Path
import torch
import numpy as np
import shutil
import matplotlib.pyplot as plt
from postprocessing.visualization import _aligned_colorbar

from extend_plumes import cut_dataset_in_pieces, prepare_dataset_for_2levels, pipeline
from main import run, save_inference
from data_stuff.utils import SettingsTraining
from preprocessing.prepare import prepare_data_and_paths


%reload_ext autoreload
%autoreload 2

# Application to small dataset

In [ ]:
dataset_prep = "dataset_2d_small_100dp inputs_gksi"
number_boxes = 4
inputs_prop = "t"
with open("paths.yaml", "r") as paths:
	paths = yaml.safe_load(paths)
	prepared1_dir = Path(paths["datasets_prepared_dir"]) / dataset_prep
	prepared_pieces_dir = Path(paths["datasets_prepared_dir"]) / f"{dataset_prep} cut_{number_boxes}pieces separate_boxes"
	prepared_pieces_dir.mkdir(parents=True, exist_ok=True)
cut_dataset_in_pieces(number_boxes, prepared1_dir, prepared_pieces_dir)
prepare_dataset_for_2levels(dataset_prep, number_boxes, "t", paths, prepared1_dir, prepared_pieces_dir)

## TODO in between: run 2 models on prior prepared datasets

# Pipeline to test the predictions of the models of both levels

In [ ]:
# dataset_prep = "dataset_2d_small_100dp inputs_gksi invert cut_4pieces separate_boxes"
# model = "dataset_100dp_invert inputs_t case_train"
dataset_prep = "dataset_2d_small_100dp inputs_gksi cut_4pieces separate_boxes"
model = "model cut_4pieces separate_boxes 2nd level t without_last_box"
for id in range(10):
    pipeline(id, dataset_prep, "t", nr_boxes_orig=4, model=model)

# Cut and test medium domain

In [5]:
# prepare dataset
dataset_prep = "dataset_medium_k_3e-10_1000dp"
inputs = "gksi"

args = {}
args["dataset_raw"] = dataset_prep
args["dataset_prep"] = ""
args["device"] = "cuda:3"
args["epochs"] = 1
args["case"] = "finetune"
args["model"] = "model cut_4pieces separate_boxes 1st level"
args["destination"] = ""
args["inputs"] = inputs
args["case_2hp"] = False
args["visualize"] = False
args["save_inference"] = False
settings = SettingsTraining(**args)

settings = prepare_data_and_paths(settings)

/home/pelzerja/pelzerja/test_nn/datasets_prepared/1hp_boxes/dataset_medium_k_3e-10_1000dp inputs_gksi


Converting:   0%|          | 0/998 [00:00<?, ?it/s]

Normalizing inputs:   0%|          | 0/998 [00:00<?, ?it/s]

Normalizing labels:   0%|          | 0/998 [00:00<?, ?it/s]

Dataset prepared (/home/pelzerja/pelzerja/test_nn/datasets_prepared/1hp_boxes/dataset_medium_k_3e-10_1000dp inputs_gksi)


In [6]:
dataset_prep = "dataset_medium_k_3e-10_1000dp inputs gksi" #dataset_small_long_1hp_100dp_interim inputs_gksi
number_boxes = 16
with open("paths.yaml", "r") as paths:
	paths = yaml.safe_load(paths)
	prepared1_dir = Path(paths["datasets_prepared_dir"]) / dataset_prep
	prepared_pieces_dir = Path(paths["datasets_prepared_dir"]) / f"{dataset_prep} cut_{number_boxes}pieces separate_boxes"
	prepared_pieces_dir.mkdir(parents=True, exist_ok=True)
cut_dataset_in_pieces(number_boxes, prepared1_dir, prepared_pieces_dir)
# not required for testing:
# prepare_dataset_for_2levels(dataset_name, number_boxes, "t", paths, prepared1_dir, prepared_pieces_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/home/pelzerja/pelzerja/test_nn/datasets_prepared/1hp_boxes/dataset_medium_k_3e-10_1000dp inputs gksi/info.yaml'

In [ ]:
for id in range(5):
    pipeline(id, "test_cut_pieces_pipeline small_long", "t", nr_boxes_orig=number_boxes)